In [2]:
import re
import csv  
import time
import requests
import numpy as np
import pandas as pd
from csv import writer
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select

In [3]:
#Get the soup object from specified url
def getSoup(url):
    try:
        source = requests.get(url).text
        soup = BeautifulSoup(source, 'lxml')
        #print(url)
        return soup
    except Exception as e:
        print(e)
        pass

In [4]:
#Gets name from soup object
def getGameName(soup):
    try:
        if((soup.find(class_ = 'apphub_AppName').get_text()) == None):
            return
        else:
            gameName = soup.find(class_ = 'apphub_AppName').get_text()
            #print(gameName)
            return gameName
    except Exception as e:
        #print(e)
        pass

In [5]:
#Gets review from soup object
def getReview(soup):
    try:
        import re
        soup = getSoup('https://store.steampowered.com/app/1474030/Dead_by_Daylight__A_Binding_of_Kin_Chapter/')        
        review = soup.find(class_ = 'summary column').get_text()
        review = re.sub('\s+',' ',review.split('(')[0])
        print(review)
        return review
    except:
        pass

In [6]:
#Gets date from soup object
def getDate(soup):
    try:
        date = soup.find(class_ = 'date').get_text()
        #print(date)
        return date
    except:
        pass

In [7]:
#Gets developer name from soup object
def developerName(soup):
    try:
        developer = soup.find(href="https://store.steampowered.com/developer/doublefine?snr=1_5_9__2000").get_text()
        return developer
    except:
        pass

In [8]:
#Gets tags from soup object
def getTags(soup):
    try:
        tags = soup.find_all(class_ = 'app_tag')
        tagArray=[]
        cleanedTag = ''
        for i in tags:
            i.findNext()
            cleanedTag = i.get_text()
            cleanedTag = cleanedTag.replace('\t','')
            cleanedTag = cleanedTag.replace('\n','')
            cleanedTag = cleanedTag.replace('\r','')
            tagArray.append(cleanedTag)
        tagArray.pop()#Scrapper takes the + so we just remove it
        #print(tagArray)
        return(tagArray)
    except:
        pass

In [9]:
#Finding the different href
def getHref(url):
    try:
        soup = getSoup(url)
        links_with_text = [a['href'] for a in soup.find_all('a', href=True) if a.text]
        #print(links_with_text)
        return links_with_text
    except:
        pass

In [10]:
#This gets all information from a specific page. I dont end up using it.
def getPageInformation(url):
    try:
        soup = getSoup(url)
        try:
            gameName = getGameName(soup)
        except:
            print("no name")
        try:
            getReview(soup)
        except:
            print("no reviews")
        try:
            getDate(soup)
        except:
            print("no date")
        try:
            developerName(soup)
        except:
            print("no developer name")
        try:
            getTags(soup)
        except:
            print("no tags")
            
    except Exception as e:
        print(e)

In [11]:
#Got this function from: https://thispointer.com/python-how-to-append-a-new-row-to-an-existing-csv-file/
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [12]:
#Steam requires user to be over 18 to access some page. This is just a way to go around it
def ageCheck(driver):
    #If steam redirects you the will redirect you to a page which first 40 char are string1
    string1 = 'https://store.steampowered.com/agecheck/'
    #We check if it requires a age check
    if(driver.current_url[0:40] == string1):
        drp=Select(driver.find_element_by_id("ageYear"))
        drp.select_by_visible_text('1995')
        try:
            button = dri|ver.find_element_by_xpath('//*[@id="app_agegate"]/div[1]/div[4]/a[1]')
        except:
            button = driver.find_element_by_xpath('//*[@id="app_agegate"]/div[1]/div[3]/a[1]')
        button.click()
    elif(driver.current_url == '#'):
        pass

In [13]:
dataBase = pd.read_csv('steam_href_user_review5.csv', encoding='latin1')
dataBase

,Href
0,https://store.steampowered.com/app/1089980/The...
1,https://store.steampowered.com/app/1229490/ULT...
2,https://store.steampowered.com/app/1144400/Sen...
3,https://store.steampowered.com/app/1055540/A_S...
4,https://store.steampowered.com/app/431730/Asep...
...,...
37555,https://store.steampowered.com/app/1154790/XII...
37556,https://store.steampowered.com/app/201510/Flat...
37557,https://store.steampowered.com/app/267600/Airp...
37558,Href


In [15]:
csvFile = 'final_Data100.csv'
#Creates a csv file that will contain all the page info
with open('final_Data100.csv', 'a') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Name', 'Review', 'Date', 'Dev name', 'Tags'])
df3 = pd.read_csv('final_Data100.csv', encoding='latin1')
df3

,Name,Review,Date,Dev name,Tags


In [16]:
def scrapePage(dataBase):
    for i in range(len(dataBase)-1):
        try:
            soup = getSoup(dataBase['Href'][i])
            url = dataBase['Href'][i]
            PATH = 'C:\Program Files (x86)\chromedriver.exe'
            driver = webdriver.Chrome(PATH)
            driver.get(url)
            ageCheck(driver)
            name = getGameName(soup)
            review = getReview(soup)
            #print(name)
            print(review)
            date = getDate(soup)
            developer = developerName(soup)
            tags = getTags(soup)
            #print(tags)
            row_content = [name, review, date, developer, tags]
            append_list_as_row('final_Data100.csv', row_content)
            driver.close()
        except:
            pass
            driver.close()

In [18]:
scrapePage(dataBase)

 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 
 Mixed 


WebDriverException: Message: chrome not reachable
  (Session info: chrome=88.0.4324.190)


In [ ]:
steamtest = pd.read_csv('final_Data.csv', encoding='latin1')

In [ ]:
steamtest